#### Imports :

In [23]:
# if package not installed, install with command:
#pip install torchxrayvision

In [47]:
import torchxrayvision as xrv

import skimage
#Image Processing for Python
#scikit-image (a.k.a. skimage) is a collection of algorithms for image processing and computer vision

import cv2

import torch
#The torch package contains data structures for multi-dimensional tensors and defines mathematical operations over these tensors.

import torch.nn.functional as F

#### Import a chest xray image

In [48]:
img = skimage.io.imread('/content/02aa804e-bde0afdd-112c0b34-7bc16630-4e384014.jpg')

#### Display image details
*the image is read as an array (type = numpy.ndarray)*

In [49]:
img

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

#### Select model to use for feature extraction
*we choose densenet121-res224-chex as the model weights name from Stanford university CheXpert models*

In [50]:

model_weights_name = "densenet121-res224-chex"

#Initialize the densefeature_embeddings with an empty list:
densefeature_embeddings = []

#Initialize the prediction_embeddings with an empty list:
prediction_embeddings = []

#Scales images to be roughly [-1024 1024]:
img = xrv.datasets.normalize(img, 255)


In [51]:
# For each image check if they are 2D arrays
if len(img.shape) > 2:
    img = img[:, :, 0]
if len(img.shape) < 2:
    print("Error: Dimension lower than 2 for image!")

In [52]:
# Add color channel for prediction
#Resize using OpenCV
img = cv2.resize(img, (224, 224), interpolation = cv2.INTER_AREA)   
img = img[None, :, :]

In [53]:
# Select model 
model = xrv.models.DenseNet(weights = model_weights_name)


In [41]:
model

XRV-DenseNet121-densenet121-res224-chex

In [54]:
# Select if you want to use CUDA support for GPU (optional as it is usually pretty fast even in CPUT)
cuda = False

In [61]:
img

tensor([[[[-1024.0001, -1024.0001, -1024.0001,  ..., -1024.0001,
           -1024.0001, -1024.0001],
          [-1024.0001, -1024.0001, -1024.0001,  ..., -1024.0001,
           -1024.0001, -1024.0001],
          [-1024.0000, -1024.0000, -1024.0000,  ..., -1024.0000,
           -1024.0000, -1024.0000],
          ...,
          [-1024.0001, -1024.0001, -1024.0001,  ..., -1024.0001,
           -1024.0001, -1024.0001],
          [-1024.0001, -1024.0001, -1024.0001,  ..., -1024.0001,
           -1024.0001, -1024.0001],
          [-1024.0001, -1024.0001, -1024.0001,  ..., -1024.0001,
           -1024.0001, -1024.0001]]]])

In [55]:
output = {}
with torch.no_grad():
    img = torch.from_numpy(img).unsqueeze(0)
    if cuda:
        img = img.cuda()
        model = model.cuda()
      
    # Extract dense features
    feats = model.features(img)
    feats = F.relu(feats, inplace=True)
    feats = F.adaptive_avg_pool2d(feats, (1, 1))
    densefeatures = feats.cpu().detach().numpy().reshape(-1)
    densefeature_embeddings = densefeatures

    # Extract predicted probabilities of considered 18 classes:
    # ['Atelectasis','Consolidation','Infiltration','Pneumothorax','Edema','Emphysema',Fibrosis',
    #  'Effusion','Pneumonia','Pleural_Thickening','Cardiomegaly','Nodule',Mass','Hernia',
    #  'Lung Lesion','Fracture','Lung Opacity','Enlarged Cardiomediastinum']
    preds = model(img).cpu()
    predictions = preds[0].detach().numpy()
    prediction_embeddings = predictions  

# Return embeddings
#return densefeature_embeddings, prediction_embeddings

In [64]:
print(preds)
len(preds)


tensor([[0.2119, 0.1532, 0.5000, 0.0455, 0.2962, 0.5000, 0.5000, 0.0979, 0.1885,
         0.5000, 0.1239, 0.5000, 0.5000, 0.5000, 0.2773, 0.4033, 0.3089, 0.1374]])


1

In [65]:
print(predictions)
len(predictions)

[0.21186149 0.15321094 0.5        0.04549948 0.29623356 0.5
 0.5        0.09788145 0.1885436  0.5        0.12390422 0.5
 0.5        0.5        0.2773131  0.4033455  0.3089283  0.13744338]


18

In [66]:
print(prediction_embeddings)
len(prediction_embeddings)

[0.21186149 0.15321094 0.5        0.04549948 0.29623356 0.5
 0.5        0.09788145 0.1885436  0.5        0.12390422 0.5
 0.5        0.5        0.2773131  0.4033455  0.3089283  0.13744338]


18

In [68]:
print(densefeature_embeddings)
len(densefeature_embeddings)

[0.         0.         0.1838883  ... 0.00537282 0.         0.10955384]


1024